<h1 style='text-align: center;'>DATA PREPARATION</h1>

### Import the Libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE

from joblib import dump

from dotenv import dotenv_values

import warnings
warnings.filterwarnings("ignore")


### Complete-case Analysis

In [2]:


DIRECTORY_PATH = dotenv_values(".env.local")["DATASET_PATH"]
df = pd.read_excel(DIRECTORY_PATH)
display(df)

,Age,Gender,Color,Transparency,Glucose,Protein,pH,Specific Gravity,WBC,RBC,Epithelial Cells,Mucous Threads,Amorphous Urates,Bacteria,Diagnosis
0,76.0,FEMALE,LIGHT YELLOW,CLEAR,NEGATIVE,NEGATIVE,5.0,1.010,1-3,0-2,OCCASIONAL,RARE,NONE SEEN,OCCASIONAL,NEGATIVE
1,9.0,MALE,DARK YELLOW,SLIGHTLY HAZY,NEGATIVE,1+,5.0,1.030,1-3,0-2,RARE,FEW,FEW,MODERATE,NEGATIVE
2,12.0,MALE,LIGHT YELLOW,SLIGHTLY HAZY,NEGATIVE,TRACE,5.0,1.030,0-3,0-2,RARE,FEW,MODERATE,RARE,NEGATIVE
3,77.0,MALE,BROWN,CLOUDY,NEGATIVE,1+,6.0,1.020,5-8,LOADED,RARE,RARE,NONE SEEN,FEW,NEGATIVE
4,29.0,FEMALE,YELLOW,HAZY,NEGATIVE,TRACE,6.0,1.025,1-4,0-2,RARE,RARE,NONE SEEN,FEW,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,0.6,MALE,YELLOW,CLEAR,NEGATIVE,NEGATIVE,6.0,1.015,1-2,0-1,RARE,FEW,NONE SEEN,RARE,NEGATIVE
1433,42.0,MALE,YELLOW,CLEAR,NEGATIVE,NEGATIVE,6.5,1.010,0-2,0-2,RARE,NONE SEEN,NONE SEEN,RARE,NEGATIVE
1434,47.0,FEMALE,DARK YELLOW,CLEAR,NEGATIVE,TRACE,6.0,1.030,2-4,0-2,MODERATE,MODERATE,NONE SEEN,RARE,NEGATIVE
1435,57.0,FEMALE,DARK YELLOW,CLEAR,NEGATIVE,TRACE,5.0,1.030,0-2,0-2,PLENTY,PLENTY,NONE SEEN,FEW,NEGATIVE


In [3]:
df.isna().sum()

Age                 1
Gender              0
Color               1
Transparency        0
Glucose             0
Protein             0
pH                  0
Specific Gravity    0
WBC                 0
RBC                 0
Epithelial Cells    0
Mucous Threads      0
Amorphous Urates    0
Bacteria            0
Diagnosis           0
dtype: int64

In [4]:
df = df.dropna()

In [5]:
df.isna().sum()

Age                 0
Gender              0
Color               0
Transparency        0
Glucose             0
Protein             0
pH                  0
Specific Gravity    0
WBC                 0
RBC                 0
Epithelial Cells    0
Mucous Threads      0
Amorphous Urates    0
Bacteria            0
Diagnosis           0
dtype: int64

### Feature Encoding

In [6]:
def custom_sort(value):
    """
        Sorting string range values, and with greater than or less than, e.g, 80-90, <50, 100> or LOADED

        Parameters
        ------------
        value : str
            the string data that needs to be sorted

        Returns
        ------------
        if ```value``` contains '>', return an integer without the sign.
        elif ```value``` is equal to LOADED or TNTC (Too Numerous to Count), return the highest floating point number.
        elif ```value``` is a range e.g 10-15, calculate the midpoint of the range by splitting the range into two number and return the midpoint
    """
    
    if value.__contains__(">"):
        return int(value.replace(">", ""))
    elif value.isalpha():
        return int(np.finfo(np.float64).max)
    else:
        start, end = map(int, value.split('-'))
        return start + (end - start) / 2

def encode_features(dataframe_, ordinal_features, nominal_features):
    """
        Encodes the nominal and ordinal features of the dataset using map function for ordinal and One-Hot Encoding for nominal.

        Parameters
        ------------
        dataframe_ : pandas.core.frame.DataFrame
            the dataframe that contains the nominal and orginal features

        ordinal_features : list, tuple, set
            a list of strings that corresponds to the column name of the ordinal features

        nominal_features : list, tuple, set
            a list of strings that corresponds to the column name of the nominal features

        Returns
        ------------
        dataframe_ : pandas.core.frame.DataFrame
            returns the preprocessed dataframe with all the ordinal and nominal features encoded
    """
    
    MAPPING_REFERENCE = {"ABSENCE_REFERENCE": {"NONE SEEN":0, 
                                               "RARE":1,
                                               "FEW":2,
                                               "OCCASIONAL":3,
                                               "MODERATE":4,
                                               "LOADED":5,
                                               "PLENTY":6},
                        
                        "Color": {"LIGHT YELLOW":0,
                                  "STRAW":1,
                                  "AMBER":2,
                                  "BROWN":3,
                                  "DARK YELLOW":4,
                                  "YELLOW":5,
                                  "REDDISH YELLOW":6,
                                  "REDDISH":7,
                                  "LIGHT RED":8,
                                  "RED":9},
                         
                        "Transparency": {"CLEAR":0,
                                         "SLIGHTLY HAZY":1,
                                         "HAZY":2,
                                         "CLOUDY":3,
                                         "TURBID":4},

                        "Protein_and_Glucose": {"NEGATIVE":0,
                                                "TRACE":1,
                                                "1+":2,
                                                "2+":3,
                                                "3+":4,
                                                "4+":5}}
    
    sorted_range_values = np.array(sorted(set(df["RBC"]) | set(df["WBC"]), key=custom_sort))
    
    for ordinal in ordinal_features:
        if ordinal in ["Epithelial Cells", "Mucous Threads", "Amorphous Urates", "Bacteria"]:
            dataframe_[ordinal] = dataframe_[ordinal].map(MAPPING_REFERENCE["ABSENCE_REFERENCE"])

        elif ordinal in ["Protein", "Glucose"]:
            dataframe_[ordinal] = dataframe_[ordinal].map(MAPPING_REFERENCE["Protein_and_Glucose"])

        elif ordinal in ["WBC", "RBC"]:
            num_of_splits = 13
            splitted_array = np.hsplit(np.array(sorted_range_values), num_of_splits)

            list_of_bin = [bin_0, bin_1, bin_2, bin_3, bin_4, bin_5, bin_6, bin_7, bin_8, bin_9, bin_10, bin_11, bin_12] = splitted_array
        
            def mapping_function(value):
                map_ = {tuple(bin_):idx for idx, bin_ in enumerate(list_of_bin)}

                for key, reference in map_.items():
                    if value in key:
                        return reference

            dataframe_[ordinal] = dataframe_[ordinal].map(mapping_function)
        else:
            dataframe_[ordinal] = dataframe_[ordinal].map(MAPPING_REFERENCE[ordinal])
            
    for nominal in nominal_features:
        dataframe_ = pd.concat([dataframe_, pd.get_dummies(dataframe_[nominal])["FEMALE"]], axis=1).drop(nominal, axis=1)
        
    return dataframe_

df = encode_features(df, 
                    ordinal_features=["Transparency", 
                                        "Epithelial Cells", 
                                        "Mucous Threads", 
                                        "Amorphous Urates", 
                                        "Bacteria",
                                        "Color",
                                        "Protein",
                                        "Glucose",
                                        "WBC",
                                        "RBC"],

                    nominal_features=["Gender"])

df

,Age,Color,Transparency,Glucose,Protein,pH,Specific Gravity,WBC,RBC,Epithelial Cells,Mucous Threads,Amorphous Urates,Bacteria,Diagnosis,FEMALE
0,76.0,0,0,0,0,5.0,1.010,0,0,3,1,0,3,NEGATIVE,True
1,9.0,4,1,0,2,5.0,1.030,0,0,1,2,2,4,NEGATIVE,False
2,12.0,0,1,0,1,5.0,1.030,0,0,1,2,4,1,NEGATIVE,False
3,77.0,3,3,0,2,6.0,1.020,3,12,1,1,0,2,NEGATIVE,False
4,29.0,5,2,0,1,6.0,1.025,1,0,1,1,0,2,NEGATIVE,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,0.6,5,0,0,0,6.0,1.015,0,0,1,2,0,1,NEGATIVE,False
1433,42.0,5,0,0,0,6.5,1.010,0,0,1,0,0,1,NEGATIVE,False
1434,47.0,4,0,0,1,6.0,1.030,1,0,4,4,0,1,NEGATIVE,True
1435,57.0,4,0,0,1,5.0,1.030,0,0,6,6,0,2,NEGATIVE,True


### Data Splitting & Feature Scaling

In [7]:
def normalize_splits(X, y, test_size_, random_state_, continuous_features, categorical_features):
    """
        it normalizes only the continous feature in the dataframe using MinMaxScaler.

        Parameters
        ------------
        X : pandas.core.frame.DataFrame
            a pandas dataframe that contains all the features

        y : pandas.core.series.Series
            a pandas series that contains the target or dependent variable

        test_size_ : float
            the test size for train_test_split

        random_state_ : int
            random state for train_test_split
            
        continuous_features: list
            a list of continous features (string) e.g ["feature1", "feature2"]

        categorical_features : list
            a list of categorical features (string) e.g ["feature1", "feature2"]

        Returns
        ------------
        scaler_ : sklearn.preprocessing._data.MinMaxScaler
            returns the training data scaler
            
        scaled_splits : tuple
            returns the dataframe with the X and y splitted and normalized

        unscaled_splits : list
            returns the dataframe with the X and y splitted and NOT normalized

    """

    scaler_ = MinMaxScaler()

    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size = test_size_,
                                                        random_state = random_state_,
                                                        stratify = y)

    scaler_.fit(X_train[continuous_features])

    X_train_scaled = scaler_.transform(X_train[continuous_features])
    X_train_scaled = pd.DataFrame(X_train_scaled, columns = continuous_features)

    X_train_scaled = pd.concat([X_train_scaled, X_train[categorical_features].reset_index(drop=True)], axis=1)

    X_test_scaled = scaler_.transform(X_test[continuous_features])
    X_test_scaled = pd.DataFrame(X_test_scaled, columns = continuous_features)

    X_test_scaled = pd.concat([X_test_scaled, X_test[categorical_features].reset_index(drop=True)], axis=1)

    y_train_encoded = y_train.map({"NEGATIVE": 0, "POSITIVE": 1})
    y_test_encoded = y_test.map({"NEGATIVE": 0, "POSITIVE": 1})

    scaled_splits = (X_train_scaled, X_test_scaled, y_train_encoded, y_test_encoded)
    unscaled_splits = [X_train, X_test, y_train_encoded, y_test_encoded]
    
    return scaler_, scaled_splits, unscaled_splits

In [8]:

X = df[["Age", "Color", "Transparency", "pH", "Glucose", "Protein", "Specific Gravity",
        "WBC", "RBC", "Epithelial Cells", "Mucous Threads",
        "Amorphous Urates", "Bacteria", "FEMALE"]]

y = df["Diagnosis"]

train_scaler, scaled_splits, unscaled_splits = normalize_splits(X, 
                                                                y, 
                                                                test_size_= 0.3, 
                                                                random_state_= 42, 
                                                                continuous_features= ["Age", "Color", "Transparency",
                                                                                      "Glucose", "Protein", "pH", 
                                                                                      "Specific Gravity", "WBC",
                                                                                      "RBC", "Epithelial Cells", "Mucous Threads",
                                                                                      "Amorphous Urates", "Bacteria"],                                                        
                                                                categorical_features= ["FEMALE"])

### Preprocessed Data

In [9]:
X_train, _, y_train, _ = scaled_splits

display(X_train)

,Age,Color,Transparency,Glucose,Protein,pH,Specific Gravity,WBC,RBC,Epithelial Cells,Mucous Threads,Amorphous Urates,Bacteria,FEMALE
0,0.684440,0.714286,0.25,0.0,0.00,0.000000,0.8,0.333333,0.000000,0.666667,0.333333,0.166667,0.6,True
1,0.336235,0.571429,0.00,0.0,0.25,0.333333,0.8,0.000000,0.000000,0.166667,0.666667,0.000000,0.0,False
2,0.162133,0.571429,0.50,0.0,0.50,0.333333,0.8,0.083333,0.000000,0.333333,0.666667,0.000000,0.6,True
3,0.173014,0.714286,0.00,0.0,0.00,0.333333,0.2,0.083333,0.000000,0.666667,0.166667,0.000000,0.0,True
4,0.510337,0.000000,0.00,0.0,0.00,0.333333,0.2,0.000000,0.000000,0.166667,0.166667,0.000000,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,0.532100,0.571429,0.25,0.0,0.00,0.500000,0.4,0.166667,0.083333,0.666667,0.166667,0.000000,0.2,False
1000,0.053319,0.571429,0.00,0.0,0.25,0.333333,0.6,0.083333,0.000000,0.166667,0.333333,0.000000,0.0,True
1001,0.031556,0.714286,0.00,0.0,0.25,0.000000,0.8,0.000000,0.000000,0.166667,0.666667,0.000000,0.0,True
1002,0.118607,0.000000,0.00,0.0,0.00,0.500000,0.0,0.000000,0.000000,0.166667,0.166667,0.000000,0.0,False


In [10]:
display(y_train)

448     0
1186    0
675     0
1252    0
467     0
       ..
1430    0
1060    0
912     0
597     0
1276    0
Name: Diagnosis, Length: 1004, dtype: int64

### Export the Data & Scaler

In [11]:
dump(train_scaler, "data/train_scaler.joblib")
dump(scaled_splits, "data/preprocessed_data.joblib")
dump(unscaled_splits, "data/preprocessed_unscaled_data.joblib")

['data/preprocessed_unscaled_data.joblib']